In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install lightfm scikit-optimize

In [73]:
# import dependent libraries
import pandas as pd
import scipy
from scipy.sparse import csr_matrix
import numpy as np
from IPython.display import Markdown, display

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
%matplotlib inline

from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from lightfm import LightFM
from skopt import forest_minimize

In [ ]:
df_res = pd.read_csv('/content/drive/My Drive/The travel Back/british_final3.csv')
df_res.head(2)

,Unnamed: 0,Unnamed: 0.1,Name,Rating,No_of_Reviews,Latitude,Longitude,day_a,day_a1,time_a,time_a1,time_a2,time_a3,day_b,day_b1,time_b,time_b1,time_b2,time_b3,day_c,day_c1,time_c,time_c1,time_c2,time_c3,day_d,day_d1,time_d,time_d1,time_d2,time_d3,day_e,day_e1,time_e,time_e1,time_e2,time_e3,day_f,time_f,time_f1,...,Ukrainian,Vietnamese,Wine Bar,Xinjiang,Yunnan,Azerbaijani,Bangladeshi,Catalan,Central-Italian,Chilean,Croatian,Dining bars,Egyptian,Hungarian,Neapolitan,Scottish,Sicilian,Southern-Italian,Kaiseki,Calabrian,Caucasian,Colombian,Czech,Northern-Italian,Romana,Romanian,Lazio,After-hours,Breakfast,Brunch,Dinner,Drinks,Lunch,None_y,After-hours,Breakfast,Brunch,Dinner,Drinks,Lunch
0,0,0,Freshii,3.5,9.0,49.036045,-122.293106,1.0,5.0,7.0,20.0,NaN,NaN,6.0,0.0,11.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,1,1,Pantry Family Restaurant,4.0,10.0,49.049360,-122.312770,0.0,4.0,6.3,21.3,NaN,NaN,5.0,6.0,6.3,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
df_res.drop(df_res.columns[7:40], axis='columns', inplace=True)
df_res


,Unnamed: 0,Unnamed: 0.1,Name,Rating,No_of_Reviews,Latitude,Longitude,res_id,City_encode,Price_code,Est_code,Food,Service,Value,Atmosphere,aburi sushi,acai bowl,adobo chicken,affogato,ahi,ahi tuna,all you can eat,almond chicken,almond croissant,aloo gobi,amazing beer,americano,amritsari fish,amritsari kulcha,amuse bouche,antipasti,apple butter,apple crumble,apple fritters,arancini,arancino,arctic char,arm,artisan cheese,arugula salad,...,Ukrainian,Vietnamese,Wine Bar,Xinjiang,Yunnan,Azerbaijani,Bangladeshi,Catalan,Central-Italian,Chilean,Croatian,Dining bars,Egyptian,Hungarian,Neapolitan,Scottish,Sicilian,Southern-Italian,Kaiseki,Calabrian,Caucasian,Colombian,Czech,Northern-Italian,Romana,Romanian,Lazio,After-hours,Breakfast,Brunch,Dinner,Drinks,Lunch,None_y,After-hours,Breakfast,Brunch,Dinner,Drinks,Lunch
0,0,0,Freshii,3.5,9.0,49.036045,-122.293106,0,3,1,5,4.0,4.5,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,1,1,Pantry Family Restaurant,4.0,10.0,49.049360,-122.312770,1,3,0,5,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
2,2,2,Townhall Public House Abbotsford,4.0,123.0,49.047710,-122.291150,2,3,0,5,3.5,4.0,3.5,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,3,3,Golden Dragon Restaurant,3.5,39.0,49.048800,-122.295150,3,3,0,5,4.0,3.5,4.0,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
4,4,4,Townline Taphouse & Grill,1.0,1.0,49.068900,-122.358580,4,3,3,5,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9430,33218,33218,Hopkins Harvest & The Hot Spot,4.5,91.0,50.468094,-115.980450,33218,224,0,5,4.5,4.5,4.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
9431,33221,33221,Edible Acres Cafe,5.0,7.0,50.467415,-115.978590,33221,224,0,5,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
9432,33279,33279,Two Scoop Steve,4.5,27.0,49.080708,-116.103870,33279,226,1,5,4.5,5.0,4.5,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
9433,33280,33280,Smoke'N'Jugz Roadhouse,4.5,19.0,49.085630,-116.089920,33280,226,0,5,4.5,5.0,4.5,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1


In [ ]:
df_res.isnull().sum()

Unnamed: 0       0
Unnamed: 0.1     0
Name             1
Rating           0
No_of_Reviews    0
                ..
 Breakfast       0
 Brunch          0
 Dinner          0
 Drinks          0
 Lunch           0
Length: 9944, dtype: int64

In [ ]:
df_user = pd.read_csv('/content/drive/My Drive/The travel Back/modified.csv')
df_user.head()

,Unnamed: 0,user_id,restaurant_id,Rating
0,0,0,10,0.0
1,1,1,68,0.0
2,2,2,68,0.0
3,3,3,68,0.5
4,4,4,10,1.0


In [ ]:
#check sparsity
n_users = df_user.user_id.unique().shape[0]
n_items = df_user.restaurant_id.unique().shape[0]

In [ ]:
print('Number of users: {}'.format(n_users))
print('Number of res: {}'.format(n_items))
print('Sparsity: {:4.3f}%'.format(float(df_user.shape[0]) / float(n_users*n_items) * 100))

Number of users: 89
Number of res: 35
Sparsity: 16.051%


In [ ]:
#model establishment
from lightfm.data import Dataset
dataset = Dataset()
dataset.fit(df_user.user_id,df_user.restaurant_id)
type(dataset)
num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 89, num_items 35.


In [ ]:
dataset.fit_partial(items=df_res.res_id,
                    item_features=['Rating'])

dataset.fit_partial(items=df_res.res_id,
                    item_features=['No_of_Reviews'])

tar_cols = [x for x in df_res.columns[11:]]


dataset.fit_partial(items=df_res.res_id,
                    item_features=tar_cols)
user_cols = ['Rating']
dataset.fit_partial(users=df_user.user_id,
                    user_features = ['Rating'])

In [ ]:
print(type(dataset))
print(dataset.model_dimensions())
print(dataset.user_features_shape())
print(dataset.item_features_shape())
#print(dataset.mapping())
print(dataset.interactions_shape())

<class 'lightfm.data.Dataset'>
(90, 19380)
(89, 90)
(9445, 19380)
(89, 9445)


In [ ]:
def create_interaction_matrix(df,user_col, item_col, rating_col, norm= False, threshold = None):
    '''
    Function to create an interaction matrix dataframe from transactional type interactions
    Required Input -
        - df = Pandas DataFrame containing user-item interactions
        - user_col = column name containing user's identifier
        - item_col = column name containing item's identifier
        - rating col = column name containing user feedback on interaction with a given item
        - norm (optional) = True if a normalization of ratings is needed
        - threshold (required if norm = True) = value above which the rating is favorable
    Expected output - 
        - Pandas dataframe with user-item interactions ready to be fed in a recommendation algorithm
    '''
    interactions = df.groupby([user_col, item_col])[rating_col] \
            .sum().unstack().reset_index(). \
            fillna(0).set_index(user_col)
    if norm:
        interactions = interactions.applymap(lambda x: 1 if x > threshold else 0)
    return interactions

In [ ]:
# Creating interaction matrix using rating data
interactions = create_interaction_matrix(df = df_user,
                                         user_col = 'user_id',
                                         item_col = 'restaurant_id',
                                         rating_col = 'Rating')


interactions.head(2)

restaurant_id,10,16,41,67,68,79,89,106,109,124,126,173,217,230,232,310,349,350,358,379,381,1149,1305,1378,1563,2111,2243,2335,3086,3395,3837,3876,3918,4317,10244
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,2.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from scipy.sparse import csr_matrix
interactions = csr_matrix(interactions.values)

In [ ]:
def build_dict(df,tar_cols,val_list):
    rst = {}
    #print(tar_cols)
    for col in tar_cols:
        rst[col] = df[col]
    #print(rst.values())
    sum_val = sum(list(rst.values())) # get sum of all the tfidf values
    
    if(sum_val == 0):
        return rst
    else:
        
        w = (2-sum(val_list))/sum_val # weight for each tag to be able to sum to 1
        for key,value in rst.items():
            rst[key] = value * w
    return rst



In [ ]:
dataset

In [ ]:
# get max of each column to regularize value to [0,1]
max_rating = max(df_res.Rating)
max_b_no_rev = max(df_res.No_of_Reviews)
print(max_rating)
print(max_b_no_rev)


# give CF info weight 0.5, all other 0.5. Then in others, give (star, review count) 0.25 and tags 0.25
item_features = dataset.build_item_features(((x['res_id'], 
                                              {'Rating':0.5*x['Rating']/max_rating,
                                               'No_of_Reviews':0.5*x['No_of_Reviews']/max_b_no_rev,
                                               **build_dict(x,tar_cols,[0.5*x['Rating']/max_rating,
                                                           0.5*x['No_of_Reviews']/max_b_no_rev])})
                                              for index,x in df_res.iterrows()))
print(repr(item_features))
print(item_features.shape)

5.0
4507.0
<9445x19380 sparse matrix of type '<class 'numpy.float32'>'
	with 93746170 stored elements in Compressed Sparse Row format>
(9445, 19380)


In [ ]:
def user_build_dict(df,tar_cols,val_list):
    rst = {}
    for col in tar_cols:
        rst[col] = df[col]
    sum_val = sum(list(rst.values())) # get sum of all the tfidf values
    if(sum_val == 0):
        return rst
    else:
        w = (2-(val_list))/sum_val # weight for each tag to be able to sum to 1
        for key,value in rst.items():
            rst[key] = value * w
    return rst

In [ ]:
max_rating_u = max(df_user.Rating)
print(max_rating)
# give CF info weight 0.5, all other 0.5. Then in others, give (star, review count) 0.25 and tags 0.25
user_features = dataset.build_user_features(((x['user_id'], 
                                              {'Rating':x['Rating']/max_rating_u,
                                               **user_build_dict(x,user_cols,x['Rating']/max_rating_u)})
                                              for index,x in df_user.iterrows()))


5.0


In [ ]:
print(repr(item_features))
print(item_features.shape)

print(repr(user_features))
print(user_features.shape)

<9445x19380 sparse matrix of type '<class 'numpy.float32'>'
	with 93746170 stored elements in Compressed Sparse Row format>
(9445, 19380)
<89x90 sparse matrix of type '<class 'numpy.float32'>'
	with 178 stored elements in Compressed Sparse Row format>
(89, 90)


In [ ]:
#check features to see if weights make sense
idx = 3
tt = list(item_features[idx].nonzero())
print(item_features[idx].shape)
print(tt)

uu = list(user_features[idx].nonzero())
print(user_features[idx].shape)
uu

(1, 19380)
[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32), array([    3,  9445,  9446,  9447,  9448,  9449, 10287, 11202, 13732,
       17110, 19207, 19332, 19370, 19379], dtype=int32)]
(1, 90)


[array([0, 0], dtype=int32), array([ 3, 89], dtype=int32)]

In [ ]:
tt = item_features[idx].todense()
import pandas as pd
tt = pd.DataFrame(tt)
print(list(map(set,tt.values)))# feature weight sum to 1, so if two are nonzero, each take weight 0.5. 

uu = user_features[idx].todense()
uu = pd.DataFrame(uu)
list(map(set,uu.values))# feature weight sum to 1, so if two are nonzero, each take weight 0.5.

[{0.0, 0.33333334, 0.11308994, 0.14999999, 0.0014791805, 0.10052439, 0.025131097}]


[{0.0, 0.11881188, 0.8811881}]

In [ ]:
tt

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,19340,19341,19342,19343,19344,19345,19346,19347,19348,19349,19350,19351,19352,19353,19354,19355,19356,19357,19358,19359,19360,19361,19362,19363,19364,19365,19366,19367,19368,19369,19370,19371,19372,19373,19374,19375,19376,19377,19378,19379
0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025131


In [ ]:
uu

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89
0,0.0,0.0,0.0,0.118812,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.881188


In [ ]:
seed = 123
from lightfm.cross_validation import random_train_test_split
train,test=random_train_test_split(interactions,test_percentage=0.2,random_state=np.random.RandomState(seed))

print('The dataset has %s users and %s items, '
      'with %s interactions in the test and %s interactions in the training set.'
      % (train.shape[0], train.shape[1], test.getnnz(), train.getnnz()))

train.multiply(test).nnz == 0 # make sure train and test are truly disjoint

The dataset has 89 users and 35 items, with 95 interactions in the test and 378 interactions in the training set.


True

In [47]:
def sample_train_recommendation(model, train, data_meta, user_ids, k, name, mapping, tag=None, user_features=None,
                                item_features=None, num_threads=2):
    n_users, n_items = train.shape

    # =============================================================================
    #     ranks = model.predict_rank(interactions,
    #                                train_interactions=train_interactions,
    #                                user_features=user_features,
    #                                item_features=item_features,
    #                                num_threads=num_threads,
    #                                check_intersections=check_intersections,
    #                                )
    #
    #     ranks.data = np.less(ranks.data, k, ranks.data)
    #
    #     precision = np.squeeze(np.array(ranks.sum(axis=1))) / k
    #
    #     if not preserve_rows:
    #         precision = precision[test_interactions.getnnz(axis=1) > 0]
    #
    #     return precision
    # =============================================================================

    for user_id in user_ids:

        t_idx = {value: key for key, value in mapping.items()}
        u_idx = [x for x in train.tocsr()[user_id].indices]
        known_positives = data_meta.loc[u_idx, name]  # may need change
        if tag is not None:
            known_tags = data_meta.loc[u_idx, tag]  # get item tags.

        if (len(known_positives) < k):
            print('not enough known positives, return max number')

        scores = model.predict(user_id, np.arange(n_items), user_features=user_features, item_features=item_features,
                               num_threads=num_threads)
        i_idx = [x for x in np.argsort(-scores)]
        top_items = data_meta.loc[i_idx, name]
        if tag is not None:
            top_tags = data_meta.loc[i_idx, tag]  # get item tags.

        print("**User %s**" % user_id)
        print("**Known positives:**")

        if tag is not None:
            for x in range(len(known_positives)):
                print(" %s | %s" % (known_positives.values[x], known_tags.values[x]))
        else:
            for x in known_positives[:len(known_positives)]:
                print("        %s" % x)

        print("**Recommended:**")
        cnt = 0
        if tag is not None:
            for x in range(k):
                print(" %s | %s" % (top_items.values[x], top_tags.values[x]))
                if (top_items.values[x] in known_positives.values):
                    cnt += 1
                    print('This one clicked')
        else:
            for x in top_items[:k]:
                print("        %s" % x)
                if (x in known_positives.values):
                    cnt += 1
                    print('This one clicked')
        #printmd('*cnt: *' + str(cnt))
        print('*k_p: %s*'%str(len(known_positives)))
        p_k = cnt / k
        print('*precicion at k : %s*'%str(p_k))
        print('----------------------------------------------------------------------')


In [ ]:

from lightfm import LightFM

# Set the number of threads; you can increase this if you have more physical cores available.

## These parameters are obtained after tunning for specific loss through skopt in the bottom chunk
NUM_THREADS = 25
NUM_COMPONENTS = 43    
NUM_EPOCHS = 18
ITEM_ALPHA = 2.88752e-6
learning_rate=0.06652
k = 5 # for precision at k

## Pure Collaborative Filtering models

# Logistic loss
model1 = LightFM(loss='logistic',random_state=seed,
                item_alpha=ITEM_ALPHA,
               no_components=NUM_COMPONENTS,
               learning_rate=learning_rate)

# time it.
%time model1 = model.fit(train,epochs=NUM_EPOCHS,num_threads=NUM_THREADS)


# Import the evaluation routines
from lightfm.evaluation import auc_score

# Compute and print the AUC score
train_auc = auc_score(model, train, num_threads=NUM_THREADS).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

# We pass in the train interactions to exclude them from predictions.
# This is to simulate a recommender system where we do not
# re-recommend things the user has already interacted with in the train
# set.


test_auc = auc_score(model1, test,num_threads=NUM_THREADS).mean()
print('Collaborative filtering test AUC: %s' % test_auc)

from lightfm.evaluation import precision_at_k,recall_at_k

print("Train precision: %.4f" % precision_at_k(model1, train, k=k,num_threads=NUM_THREADS).mean())
print("Test precision: %.4f" % precision_at_k(model1, test,train_interactions=train, k=k,num_threads=NUM_THREADS).mean())


CPU times: user 20.1 ms, sys: 7 ms, total: 27.1 ms
Wall time: 18.4 ms
Collaborative filtering train AUC: 0.6894016
Collaborative filtering test AUC: 0.6284611
Train precision: 0.2404
Test precision: 0.0730


In [48]:
sample_train_recommendation(model1,train,df_res,range(10),5,'Name',mapping=dataset.mapping()[2])

not enough known positives, return max number
**User 0**
**Known positives:**
        Boston Pizza
**Recommended:**
        Common Place Cafe
        Pizza Hut
        Maru Sushi
        Ricky's Country Restaurant
        Antonio's Restaurant
*k_p: 1*
*precicion at k : 0.0*
----------------------------------------------------------------------
not enough known positives, return max number
**User 1**
**Known positives:**
        Station House Bar and Grill
        Tony's Place Restaurant
        New York Fries
        Big Feast Bistro & Retail
**Recommended:**
        Common Place Cafe
        Pizza Hut
        Maru Sushi
        Ricky's Country Restaurant
        Antonio's Restaurant
*k_p: 4*
*precicion at k : 0.0*
----------------------------------------------------------------------
not enough known positives, return max number
**User 2**
**Known positives:**
        Tony's Place Restaurant
        Yummy Sushi Gallery
        Brgr Brgr
        Antonio's Restaurant
**Recommended:**
  

In [ ]:
NUM_THREADS = 25
NUM_COMPONENTS = 45    
NUM_EPOCHS = 43
ITEM_ALPHA = 1.3846e-6
learning_rate=0.0161445

# BPR loss
model2 = LightFM(loss='bpr',random_state=seed,
                item_alpha=ITEM_ALPHA,
               no_components=NUM_COMPONENTS,
               learning_rate=learning_rate)

# time it.
%time model2 = model2.fit(train,epochs=NUM_EPOCHS,num_threads=NUM_THREADS)


# Compute and print the AUC score
train_auc = auc_score(model2, train, num_threads=NUM_THREADS).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

test_auc = auc_score(model2, test,num_threads=NUM_THREADS).mean()
print('Collaborative filtering test AUC: %s' % test_auc)

print("Train precision: %.4f" % precision_at_k(model2, train, k=k,num_threads=NUM_THREADS).mean())
print("Test precision: %.4f" % precision_at_k(model2, test,train_interactions=train, k=k,num_threads=NUM_THREADS).mean())

In [49]:
sample_train_recommendation(model2,train,df_res,range(10),5,'Name',mapping=dataset.mapping()[2])

not enough known positives, return max number
**User 0**
**Known positives:**
        Boston Pizza
**Recommended:**
        Common Place Cafe
        Pizza Hut
        Maru Sushi
        Ricky's Country Restaurant
        Antonio's Restaurant
*k_p: 1*
*precicion at k : 0.0*
----------------------------------------------------------------------
not enough known positives, return max number
**User 1**
**Known positives:**
        Station House Bar and Grill
        Tony's Place Restaurant
        New York Fries
        Big Feast Bistro & Retail
**Recommended:**
        Common Place Cafe
        Pizza Hut
        Maru Sushi
        Ricky's Country Restaurant
        Antonio's Restaurant
*k_p: 4*
*precicion at k : 0.0*
----------------------------------------------------------------------
not enough known positives, return max number
**User 2**
**Known positives:**
        Tony's Place Restaurant
        Yummy Sushi Gallery
        Brgr Brgr
        Antonio's Restaurant
**Recommended:**
  

In [ ]:
NUM_THREADS = 25
NUM_COMPONENTS = 21    
NUM_EPOCHS = 16
ITEM_ALPHA = 5.97967e-6
learning_rate=0.033

# Let's fit a WARP model
model3 = LightFM(loss='warp',random_state=seed,
                item_alpha=ITEM_ALPHA,
               no_components=NUM_COMPONENTS,
               learning_rate=learning_rate)

#  time it.
%time model3 = model3.fit(train,epochs=NUM_EPOCHS,num_threads=NUM_THREADS)



# Compute and print the AUC score
train_auc = auc_score(model3, train, num_threads=NUM_THREADS).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

test_auc = auc_score(model3, test,num_threads=NUM_THREADS).mean()
print('Collaborative filtering test AUC: %s' % test_auc)


print("Train precision: %.4f" % precision_at_k(model3, train, k=k,num_threads=NUM_THREADS).mean())
print("Test precision: %.4f" % precision_at_k(model3, test,train_interactions=train, k=k,num_threads=NUM_THREADS).mean())


In [ ]:
sample_train_recommendation(model3,train,df_res,range(10),5,'Name',mapping=dataset.mapping()[2])

In [50]:
NUM_THREADS = 25
NUM_COMPONENTS = 30
learning_rate = 0.0053
NUM_EPOCHS = 6
ITEM_ALPHA = 2.228e-5

## Hybrid models

#Combine user_feature and item feature

#logistic loss
model.iii = LightFM(loss='logistic',
                item_alpha=ITEM_ALPHA,
                    random_state=seed,
               no_components=NUM_COMPONENTS)

# time it.
%time model.iii = model.iii.fit(train,user_features=user_features,item_features=item_features,epochs=NUM_EPOCHS,num_threads=NUM_THREADS)

# Compute and print the AUC score
train_auc = auc_score(model.iii, train,user_features=user_features,item_features=item_features, num_threads=NUM_THREADS).mean()
print(' Hybrid train AUC: %s' % train_auc)

test_auc = auc_score(model.iii, test,user_features=user_features,item_features=item_features,num_threads=NUM_THREADS).mean()
print('Hybrid test AUC: %s' % test_auc)

#precision @k
print("Train precision: %.4f" % precision_at_k(model.iii, train,
                                               item_features=item_features,user_features=user_features, k=k,
                                               num_threads=NUM_THREADS).mean())
print("Test precision: %.4f" % precision_at_k(model.iii, test,train_interactions=train,
                                              item_features=item_features,user_features=user_features, k=k,
                                             num_threads=NUM_THREADS).mean())


CPU times: user 10.1 s, sys: 13.8 ms, total: 10.1 s
Wall time: 5.23 s
 Hybrid train AUC: 0.59566426
Hybrid test AUC: 0.5315442
Train precision: 0.1820
Test precision: 0.0444


In [69]:
sample_train_recommendation(model.iii,train,df_res,[7],10,'Name',mapping=dataset.mapping()[2],tag='Pizza',
                              user_features = user_features,item_features=item_features)


not enough known positives, return max number
**User 7**
**Known positives:**
 Station House Bar and Grill | 0
 Tony's Place Restaurant | 0
 New York Fries | 0
 Big Feast Bistro & Retail | 0
**Recommended:**
 Station House Bar and Grill | 0
This one clicked
 Pizza Hut | 1
 Townline Taphouse & Grill | 0
 Maru Sushi | 0
 Ricardo's Pizza | 0
 Big Feast Bistro & Retail | 0
This one clicked
 Freshii | 0
 Common Place Cafe | 0
 Golden Dragon Restaurant | 0
 Paliotti's Italian Restaurants | 0
*k_p: 4*
*precicion at k : 0.2*
----------------------------------------------------------------------


**User 7**


**Known positives:**
        
        Station House Bar and Grill
        Tony's Place Restaurant
        New York Fries
        Big Feast Bistro & Retail



**Recommended:**

        Common Place Cafe
        Pizza Hut
        Maru Sushi
        Ricky's Country Restaurant
        Antonio's Restaurant

**User 7**

**Known positives:**

    Station House Bar and Grill | 0
    Tony's Place Restaurant | 0
    New York Fries | 0
    Big Feast Bistro & Retail | 0

**Recommended:**

    Station House Bar and Grill | 0
    This one clicked
    Pizza Hut | 1
    Townline Taphouse & Grill | 0
    Maru Sushi | 0
    Ricardo's Pizza | 0
    Big Feast Bistro & Retail | 0
    This one clicked
    Freshii | 0
    Common Place Cafe | 0
    Golden Dragon Restaurant | 0
    Paliotti's Italian Restaurants | 0

In [71]:
def get_user_index(test):
    return scipy.sparse.find(test)[0]

In [77]:
def sample_test_recommendation(model, train, test, data_meta, user_ids, k, name, mapping, tag=None,
                               train_interactions=None, user_features=None,
                               item_features=None, num_threads=2):
    n_users, n_items = test.shape

    for user_id in user_ids:
        
        print("**User %s**" % user_id)
        
        t_idx = {value: key for key, value in mapping.items()}
        u_idx = [x for x in test.tocsr()[user_id].indices]

        known_positives = data_meta.loc[u_idx, name]  # may need change

        print('length of known_positives: ' + str(len(known_positives)))
        if (len(known_positives) == 0):
            sample_train_recommendation(model, train, data_meta, [user_id], k, name, mapping, tag, user_features,
                                        item_features)
            continue

        elif (len(known_positives) < k):
            print('not enough known positives, return max number')

        if tag is not None:
            known_tags = data_meta.loc[u_idx, tag]  # get item tags.

        if (train_interactions is None):
            scores = model.predict(user_id, np.arange(n_items), user_features=user_features,
                                   item_features=item_features,
                                   num_threads=num_threads)
            i_idx = [x for x in np.argsort(-scores)]
            top_items = data_meta.loc[i_idx, name]
            if tag is not None:
                top_tags = data_meta.loc[i_idx, tag]  # get item tags.

        else:
            item_ids = np.delete(np.arange(n_items), train.tocsr()[user_id].indices)
            scores = model.predict(user_id, item_ids, user_features=user_features, item_features=item_features,
                                   num_threads=num_threads)
            i_idx = [x for x in np.argsort(-scores)]
            top_items = data_meta.loc[i_idx, name]
            if tag is not None:
                top_tags = data_meta.loc[i_idx, tag]  # get item tags.

        
        print("**Known positives:**")

        if tag is not None:
            for x in range(len(known_positives)):
                print(" %s | %s" % (known_positives.values[x], known_tags.values[x]))
        else:
            for x in known_positives[:len(known_positives)]:
                print("        %s" % x)

        print("**Recommended:**")
        cnt = 0
        if tag is not None:
            for x in range(k):
                print(" %s | %s" % (top_items.values[x], top_tags.values[x]))
                if (top_items.values[x] in known_positives.values):
                    cnt += 1
                    print('This one clicked')
        else:
            for x in top_items[:k]:
                print("        %s" % x)
                if (x in known_positives.values):
                    cnt += 1
                    print('This one clicked')
        #printmd('*cnt: *' + str(cnt))
        print('*k_p: %s*'%str(len(known_positives)))
        p_k = cnt / k
        print('*precicion at k : %s*'%str(p_k))
        print('----------------------------------------------------------------------')
        

In [80]:
user_index=list(set(get_user_index(test)))
sample_test_recommendation(model.iii,train,test,df_res,[user_index[5]],5,'Name',mapping=dataset.mapping()[2],
                              train_interactions=train,user_features = user_features,item_features=item_features)


**User 9**
length of known_positives: 1
not enough known positives, return max number
**Known positives:**
        Station House Bar and Grill
**Recommended:**
        The Pantry
        Yummy Sushi Gallery
        Ricky's Country Restaurant
        J's Pizza
        Freshii
*k_p: 1*
*precicion at k : 0.0*
----------------------------------------------------------------------


In [99]:
# get similar tags
def get_similar_tags(model, tag_id,k):
    # Define similarity as the cosine of the angle
    # between the tag latent vectors

    # Normalize the vectors to unit length
    tag_embeddings = (model.item_embeddings[10095:].T
                      / np.linalg.norm(model.item_embeddings[10095:], axis=1)).T

    query_embedding = tag_embeddings[tag_id]
    similarity = np.dot(tag_embeddings, query_embedding)
    most_similar = np.argsort(-similarity)[1:k+1]

    return most_similar


tag_labels = list(dataset.mapping()[3].keys())[10095:]


target_ls = ['Chinese']
for tag in target_ls:
    tag_id = tag_labels.index(tag)
    print('Most similar tags for %s: %s' % (tag_labels[tag_id],
                                            [tag_labels[x] for x in get_similar_tags(model.iii, tag_id,5)]))


Most similar tags for Chinese: [' prawn ceviche ', ' pumpkin spice ', ' guatemalan ', ' makanek ', ' selection of beer on tap ']


In [93]:
list(dataset.mapping()[1])

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 'Rating']